# HiPerCluster

### Table of Contents

- [**Step 1 Synthetic Fe-Cr APT Data Generator**](#step1)  
  
- [**Step 2 Data Preprocessing**](#step2)  
  
- [**Step 3 Model Training**](#step3)  
  
- [**Step 4 Inference**](#step4)  
  
- [**Step 5 Clustering**](#step5)  
  
- [**Step 6 Postprocessing**](#step6)  

### Step 1 Synthetic Fe-Cr APT Data Generator <a class="anchor" id="step1"></a>

In [24]:
from __future__ import annotations
import os
import math
import warnings
import numpy as np
from typing import Tuple, List
from scipy.io import savemat
from scipy.spatial.distance import pdist

try:
    from tqdm import trange
except ImportError:
    def trange(*args, **kwargs):
        return range(*args, **kwargs)


# ===================== USER INPUT SECTION =====================
Dim = np.array([80.0, 80.0, 80.0]) # Volume dimensions [X, Y, Z] in nm
TotalNumPoints = int(7000000) # Total number of atoms (Fe + Cr)
CrPercentage = 0.12 # Chromium atomic fraction
    
Density = 9.5e23 # Atom density in atoms/m^3 (unused)
DensityError = 0.2e23 # Density uncertainty (unused)
    
NumClusters = 100
# clusterInfo: [NumClusters, rx_mean, rx_sigma, ry_mean, ry_sigma, rz_mean, rz_sigma]
clusterInfo = np.array([NumClusters, 1.5, 0.2, 1.5, 0.2, 1.5, 0.1], dtype=float)
    
MinClusterSeparation = 5.0 # Minimum separation between cluster centers (nm)
OutputFolder = './SyntheticData'
os.makedirs(OutputFolder, exist_ok=True)
    
# Derived values
TotalNumCr = int(round(CrPercentage * TotalNumPoints))
TotalNumFe = TotalNumPoints - TotalNumCr
    
# Extra knobs
RANDOM_SEED = 42
F_CLUSTERED = 0.70
MAX_CENTER_TRIES = 1000000
np.random.seed(RANDOM_SEED)

# ===================== FUNCTIONS =====================
    
def generate_cluster_sizes(clusterInfo: np.ndarray) -> Tuple[np.ndarray, int, float]:
    ci = np.asarray(clusterInfo, dtype=float)
    if ci.ndim == 1:
        ci = ci[None, :]
    if ci.shape[1] != 7:
        raise ValueError("clusterInfo must have 7 values: [count, baseX, deltaX, baseY, deltaY, baseZ, deltaZ]")
            
    all_dims = []
    for row in ci:
        count = int(round(row[0]))
        if count <= 0:
                continue
        baseX, deltaX, baseY, deltaY, baseZ, deltaZ = row[1:]
        sizeX = baseX + (np.random.rand(count, 1) * 2 * deltaX) - deltaX
        sizeY = baseY + (np.random.rand(count, 1) * 2 * deltaY) - deltaY
        sizeZ = baseZ + (np.random.rand(count, 1) * 2 * deltaZ) - deltaZ
        all_dims.append(np.hstack([sizeX, sizeY, sizeZ]))
                    
    clusterDims = np.vstack(all_dims) if all_dims else np.empty((0, 3), dtype=float)
    numClusters = clusterDims.shape[0]
    maxRadius = float(np.max(clusterDims)) if numClusters > 0 else 0.0
    return clusterDims, numClusters, maxRadius


def generateClusterCenters(xMin, xMax, yMin, yMax, zMin, zMax, numClusters, sepFactor, maxRadius):
    xMin *= 0.8; xMax *= 0.8
    yMin *= 0.8; yMax *= 0.8
    zMin *= 0.8; zMax *= 0.8
                            
    candidateCount = int(1e6)
    xLower, xUpper = xMin + maxRadius, xMax - maxRadius
    yLower, yUpper = yMin + maxRadius, yMax - maxRadius
    zLower, zUpper = zMin + maxRadius, zMax - maxRadius
                            
    if not (xLower < xUpper and yLower < yUpper and zLower < zUpper):
        raise ValueError("Invalid bounds after applying maxRadius buffer.")
                                
    randX = (xUpper - xLower) * np.random.rand(candidateCount) + xLower
    randY = (yUpper - yLower) * np.random.rand(candidateCount) + yLower
    randZ = (zUpper - zLower) * np.random.rand(candidateCount) + zLower
                                
    selectedX, selectedY, selectedZ = [randX[0]], [randY[0]], [randZ[0]]
    minAllowedDist = sepFactor * maxRadius
                                
    for i in range(1, candidateCount):
        dx = np.array(selectedX) - randX[i]
        dy = np.array(selectedY) - randY[i]
        dz = np.array(selectedZ) - randZ[i]
        distances = np.sqrt(dx*dx + dy*dy + dz*dz)
        if np.min(distances) >= minAllowedDist:
            selectedX.append(randX[i])
            selectedY.append(randY[i])
            selectedZ.append(randZ[i])
            if len(selectedX) >= numClusters:
                break
                                            
    if len(selectedX) < numClusters:
        raise RuntimeError("Not enough space for requested clusters.")
                                                
    idx = np.random.permutation(len(selectedX))[:numClusters]
    centers = np.column_stack([np.array(selectedX)[idx], np.array(selectedY)[idx], np.array(selectedZ)[idx]])
                                                
    minDistance = np.min(pdist(centers)) if len(centers) >= 2 else np.inf
    return centers, minDistance


def generate_cr_clusters(Centers, Radii, CrPercentage, TotalPoints):
    total_cr = int(round(CrPercentage * TotalPoints))
    n_clusters = Centers.shape[0]
    max_per_cluster = total_cr // n_clusters
    if max_per_cluster <= 0:
        return np.empty((0, 4)), 0
                                                        
    pts_list = []
    cr_used = 0
    for i in range(n_clusters):
        rx, ry, rz = Radii[i]
        num_points = max_per_cluster
        xyz = np.hstack([np.random.randn(num_points, 1) * rx + Centers[i, 0],
                         np.random.randn(num_points, 1) * ry + Centers[i, 1],
                         np.random.randn(num_points, 1) * rz + Centers[i, 2]])
        labels = np.full((num_points, 1), i + 1, dtype=float)
        pts_list.append(np.hstack([xyz, labels]))
        cr_used += num_points
                                                            
    return np.vstack(pts_list), cr_used
                                                            

def generateBackground(Dim, Centers, dmax, NumCr, NumFe):
    total = NumCr + NumFe
    pts = np.random.rand(total, 3) * Dim
    keep = np.ones(total, dtype=bool)
    for c in Centers:
        d = np.sqrt(np.sum((pts - c)**2, axis=1))
        keep &= (d > dmax)
    pts = pts[keep, :]
                                                                
    if pts.shape[0] < total:
        warnings.warn("Not enough background space. Reducing background atoms.")
        total = pts.shape[0]
        NumCr = int(round((NumCr / (NumCr + NumFe)) * total))
        NumFe = total - NumCr
                                                                    
    return pts[:NumCr, :], pts[NumCr:NumCr + NumFe, :]


# ===================== MAIN PIPELINE =====================
if __name__ == "__main__":
    print("Generating cluster sizes...")
    Radii, numClusters, maxRadius = generate_cluster_sizes(clusterInfo)

    print("Generating cluster centers...")
    Centers, minDist = generateClusterCenters(0, Dim[0], 0, Dim[1], 0, Dim[2],
    numClusters, MinClusterSeparation, maxRadius)
    print(f"Centers generated: {Centers.shape[0]} (min distance = {minDist:.2f} nm)")

    print("Generating Cr clusters...")
    CrPoints_clusters, cr_used = generate_cr_clusters(Centers, Radii, F_CLUSTERED * CrPercentage, TotalNumPoints)
    print(f"Cr points in clusters: {CrPoints_clusters.shape[0]}")

    remainingCr = TotalNumCr - cr_used
    print(f"Remaining Cr for background: {remainingCr}")

    print("Generating background points...")
    Cr_bg, Fe_bg = generateBackground(Dim, Centers, maxRadius, remainingCr, TotalNumFe)
    print(f"Background: Cr = {Cr_bg.shape[0]}, Fe = {Fe_bg.shape[0]}")

    # Combine data
    all_points = {
        'Cr_clusters': CrPoints_clusters,
        'Cr_background': Cr_bg,
        'Fe_background': Fe_bg,
        'Centers': Centers,
        'Radii': Radii,
        'Dim': Dim
    }

    # Save as .mat file
    output_file = os.path.join(OutputFolder, "synthetic_clusters.mat")
    savemat(output_file, all_points)
    print(f"Data saved to {output_file}")
    
    # SAve Cr_clusters seperately as a .txt file
    cr_clusters_file=os.path.join(OutputFolder,"Cr_clusters.txt")
    np.savetxt(cr_clusters_file,CrPoints_clusters,fmt='%.6f')

    # Optional: Visualization (sample only for speed)
#     fig = plt.figure(figsize=(8, 6))
#     ax = fig.add_subplot(111, projection='3d')
#     ax.scatter(CrPoints_clusters[:2000, 0], CrPoints_clusters[:2000, 1], CrPoints_clusters[:2000, 2], c='r', s=1, label='Cr Clusters')
#     ax.scatter(Cr_bg[:2000, 0], Cr_bg[:2000, 1], Cr_bg[:2000, 2], c='g', s=1, label='Cr Background')
#     ax.scatter(Fe_bg[:2000, 0], Fe_bg[:2000, 1], Fe_bg[:2000, 2], c='b', s=1, label='Fe Background')
#     ax.set_xlim(0, Dim[0]); ax.set_ylim(0, Dim[1]); ax.set_zlim(0, Dim[2])
#     ax.legend()
#     plt.title("Synthetic Atom Distribution (Sample)")
#     plt.show()

Generating cluster sizes...
Generating cluster centers...
Centers generated: 100 (min distance = 8.55 nm)
Generating Cr clusters...
Cr points in clusters: 588000
Remaining Cr for background: 252000
Generating background points...


/tmp/ipykernel_845325/3694419827.py:143: UserWarning: Not enough background space. Reducing background atoms.
  warnings.warn("Not enough background space. Reducing background atoms.")


Background: Cr = 250992, Fe = 6135358
Data saved to ./SyntheticData/synthetic_clusters.mat


### Step 2 Data Preprocessing <a class="anchor" id="step2"></a>

In [25]:
# Synthetic Data Subvolume Extraction and Image Generation (Python)
# ---------------------------------------------------------------

import os
import math
import numpy as np
from PIL import Image
from matplotlib import cm
from matplotlib import colors
from scipy.io import loadmat


# ============================= USER INPUTS =============================
base_folder = "./SyntheticData"   # <-- change to your base folder
output_folder = "Train"                           # <-- folder to save generated images
num_synthetic_files = 1                           # <-- total number of synthetic datasets
cube_size = 20.0                                  # <-- subvolume size in nm
grid_size = 100                                   # <-- grid size for projection images
min_points_threshold = 300                        # <-- minimum number of points in subvolume to process
min_cluster_size = 100                            # <-- minimum cluster size to consider
resize_image_size = (100, 100)                    # <-- final image size (pixels) (width, height)

# ======================================================================
 
os.makedirs(output_folder, exist_ok=True)
 
iter_counter = 1
parameters = []          # to mirror MATLAB "parameter" cell array (stores cluster_num)
groundtruth = []         # rows: [syn, i, j, k, cluster_num]
 
def count_rgb(img: np.ndarray, out_size=(100, 100)) -> Image.Image:
    # """
    # Emulates MATLAB's imagesc + current colormap -> RGB, then resizes.
    # - Uses matplotlib's 'viridis' by default (stable, perceptually uniform).
    # - Scales to [0, max(img)], handling the all-zero case.
    # """
    img = np.asarray(img, dtype=float)
    vmax = float(img.max()) if img.size > 0 else 0.0
    if vmax <= 0.0:
        # All zeros -> produce a black image
        rgba = np.zeros((img.shape[0], img.shape[1], 4), dtype=np.float32)
    else:
        norm = colors.Normalize(vmin=0.0, vmax=vmax, clip=True)
        cmap = cm.get_cmap("viridis")  # pick a fixed, nice colormap
        rgba = cmap(norm(img))         # floats in [0,1], shape (H,W,4)
 
    rgb = (rgba[..., :3] * 255.0).astype(np.uint8)
    pil_img = Image.fromarray(rgb, mode="RGB")
    # MATLAB imresize default is bicubic-like; Pillow.BILINEAR is a reasonable match
    return pil_img.resize(out_size, resample=Image.BILINEAR)


for syn in range(1, num_synthetic_files + 1):
#     folder_name = f"GeneratedDatasetResults_Synthetic{syn}"
#     file_name = f"Synthetic{syn}_CuAtomsInClusterandSolidSolution.txt"
   file_name='Cr_clusters.txt'
   path = os.path.join(base_folder, file_name)


   # Read the dataset (expecting numeric columns; 5th col is label)
   # If your files have headers or different delimiters, adjust loadtxt arguments.
   Data = np.loadtxt(path)
 

   # Work only with spatial columns for cube tiling
   mins = Data[:, :3].min(axis=0)
   maxs = Data[:, :3].max(axis=0)
   spans = maxs - mins

   # Number of cubes along x, y, z
   num_cubes = np.floor(spans / cube_size).astype(int)  # shape (3,)

   # Guard: if any dimension has 0 cubes, nothing to do
   if np.any(num_cubes <= 0):
       continue

   # Divide dataset into subvolumes 
   for i in range(1, num_cubes[0] + 1):
       x0 = mins[0] + cube_size * (i - 1)
       x1 = x0 + cube_size
       in_x = (Data[:, 0] >= x0) & (Data[:, 0] <= x1)

       for j in range(1, num_cubes[1] + 1):
           y0 = mins[1] + cube_size * (j - 1)
           y1 = y0 + cube_size
           in_y = (Data[:, 1] >= y0) & (Data[:, 1] <= y1)

           for k in range(1, num_cubes[2] + 1):
               z0 = mins[2] + cube_size * (k - 1)
               z1 = z0 + cube_size
               in_z = (Data[:, 2] >= z0) & (Data[:, 2] <= z1)

               gt = Data[in_x & in_y & in_z]

               # Remove invalid points (label = -1) — assumes labels are in column 5 (0-based idx 4)
               if gt.size == 0:
                   continue
               gt = gt[gt[:, 2] != -1]

               if gt.shape[0] <= min_points_threshold:
                   continue

               # Count clusters >= min_cluster_size
               labels = gt[:, 3].astype(int)
               # Equivalent to MATLAB: [uniqueVals, ~, idx] + accumarray
               unique_vals, counts = np.unique(labels, return_counts=True)
               t_filtered = counts[counts >= min_cluster_size]
               cluster_num = int(t_filtered.size)

               # Scale coordinates to [0, grid_size - 1]
               sub_min = gt[:, :3].min(axis=0)
               sub_max = gt[:, :3].max(axis=0)
               ranges = sub_max - sub_min
               # avoid division by zero for flat ranges
               ranges[ranges == 0] = 1.0
               scaled = (gt[:, :3] - sub_min) / ranges * (grid_size - 1)

               # Integer indices (MATLAB used floor + 1; here we keep 0-based)
               idxs = np.floor(scaled).astype(int)
               # clamp just in case of numerical edge (e.g., exactly grid_size)
               np.clip(idxs, 0, grid_size - 1, out=idxs)

               # Initialize projections
               Count_xy = np.zeros((grid_size, grid_size), dtype=np.int32)
               Count_xz = np.zeros((grid_size, grid_size), dtype=np.int32)
               Count_yz = np.zeros((grid_size, grid_size), dtype=np.int32)

               # Populate projections (vectorized with np.add.at)
               ix, iy, iz = idxs[:, 0], idxs[:, 1], idxs[:, 2]
               np.add.at(Count_xy, (ix, iy), 1)
               np.add.at(Count_xz, (ix, iz), 1)
               np.add.at(Count_yz, (iy, iz), 1)

               # Save images (three views)
               for mat in (Count_xy, Count_xz, Count_yz):
                   img = count_rgb(mat, out_size=resize_image_size)
                   out_path = os.path.join(output_folder, f"{iter_counter}.jpg")
                   img.save(out_path, format="JPEG", quality=95)

                   parameters.append([cluster_num])
                   groundtruth.append([syn, i, j, k, cluster_num])
                   iter_counter += 1
               

# (Optional) Save metadata like MATLAB variables would
# np.savetxt(os.path.join(output_folder, "parameters.csv"), np.array(parameters, dtype=int), fmt="%d", delimiter=",")
# np.savetxt(os.path.join(output_folder, "groundtruth.csv"), np.array(groundtruth, dtype=int), fmt="%d", delimiter=",")

print(f"Done. Wrote {iter_counter-1} images to: {output_folder}")

AttributeError: module 'matplotlib.cm' has no attribute 'get_cmap'

### Step 3 Model Training <a class="anchor" id="step3"></a>

In [ ]:
# ============================================
#  Model training on ConvNeXt-Tiny
# ============================================

import numpy as np
from scipy.io import loadmat
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
import tensorflow as tf
import h5py

import tensorflow
tensorflow.test.is_gpu_available()

In [ ]:
# Load the .mat file
file = h5py.File('Train.mat', 'r')

# Extract datasets
XTrain = file['X_normalized'][:]  # Shape: (200, 200,200, 1, n)
YTrain = file['Y_img'][:]  # Shape: (n, 2)

XTrain_permuted=np.transpose(XTrain,(0,3,4,2,1))
YTrain_permuted=np.transpose(YTrain,(1,0))

print(XTrain_permuted.shape)
print(YTrain_permuted.shape)

In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
#import tensorflow_hub as hub

import datetime

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the TensorBoard extension
data = XTrain_permuted
data = tf.squeeze(data, axis=4)
print(data.shape)

# Create the data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.01,
    height_shift_range=0.01,
    horizontal_flip=True,
    vertical_flip=True
)

In [ ]:
# Ensure data and YTrain_permuted are numpy arrays
data = np.array(data)
YTrain_permuted = np.array(YTrain_permuted)

# Define the split ratio
split_ratio = 0.2
split_index = int((1 - split_ratio) * len(data))

# Shuffling the dataset before splitting
indices = np.random.permutation(len(data))
train_indices = indices[:split_index]
val_indices = indices[split_index:]

train_data = data[train_indices]
train_labels = YTrain_permuted[train_indices]
val_data = data[val_indices]
val_labels = YTrain_permuted[val_indices]

In [ ]:
import numpy as np
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras import layers, models, Input
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Resizing,GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Load pre-trained ConvNeXtTiny model without the top layer
base_model = ConvNeXtTiny(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable=True

# Create full model for fine-tunin
model = Sequential()
model.add(Resizing(224, 224, input_shape=(100,100,3)))
model.add(base_model)
model.add(GlobalAveragePooling2D())


model.add(BatchNormalization())
model.add(Dense(128,activation='relu', kernel_regularizer=l2(1e-6)))
model.add(Dropout(0.15))
model.add(Dense(1,kernel_regularizer=l2(1e-6)))


# Use a small learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)


lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Compile the model
model.compile(optimizer=opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

# Setup early stopping to prevent over-fitting.
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)



In [ ]:
# Fit the model using the data augmentation generator
import time
start_time=time.time()
history = model.fit(
    datagen.flow(train_data, train_labels, batch_size=32),
    epochs=100,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping,lr_reducer]
)
end_time=time.time()
print(f"Training time: {end_time-start_time:.2f} seconds")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss and accuracy values
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot loss on the left y-axis
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='tab:blue')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:blue')
if 'val_loss' in history.history:
    ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:cyan')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.legend(loc='upper left')
ax1.grid(True)

# Title and layout
plt.title('Model Loss and Accuracy')
fig.tight_layout()

# Save the plot as an image file
plt.savefig('Convtiny.png')

# Show the plot
plt.show()


In [ ]:
history_data = history.history
import scipy.io
scipy.io.savemat('Convtiny_history.mat', history_data)

model.save('ConvTiny.keras')

### Step 4 Inference <a class="anchor" id="step4"></a>

In [ ]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.saving import register_keras_serializable

# Path to the folder containing .jpg images
image_folder = './'
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')])

# Load all images into a numpy array
X = np.array([
    np.array(Image.open(os.path.join(image_folder, fname)).resize((100, 100)))
    for fname in image_files
])
X = X / 255.0  # if your model expects normalized input

# Load the model and specify the custom objects
loaded_model = load_model('ConvTiny.keras') 
# OR
loaded_model = load_model('ResNet50.keras')

# Predict
predictions = loaded_model.predict(X)
print(predictions)

import scipy.io
scipy.io.savemat('predictions.mat', {'predictions': predictions})

### Step 5 Clustering <a class="anchor" id="step5"></a>

In [ ]:
# ============================================
#  K-Means + HDBSCAN Clustering Script (Python)
#  Description:
#    - Loads raw 3D coordinates and model predictions 
#    - Preliminary clustering with K-Means
#    - HDBSCAN refinement
# ============================================

import os
import sys
import math
import json
import numpy as np
import subprocess
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
import h5py
from scipy.io import loadmat
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist


# ================================
# 1) USER INPUTS
# ================================
rawDataFile      = "subvolume.txt"               # <-- Raw 3D coordinates .mat; expects a variable containing Nx3 coords
predictionFile   = "predictions.mat"   # <-- .mat with model predictions (vector or scalar)
threshold        = 2.0                      # <-- distance threshold for assigning clusters
minPersistence   = 0.06                     # <-- persistence cutoff for HDBSCAN filtering


# Choose HDBSCAN mode:
HDBSCAN_MODE     = "native"                 # "native" (uses hdbscan lib) or "external" (call your script)


# External-script settings (used only if HDBSCAN_MODE == "external")
# pythonEnvPath    = "/home/tangy/miniforge3/envs/APT/bin/python"  # path to python exe
# hdbscanScript    = "hdbscanImanSecond_9.py"                      # your external script
tempDataFile     = "TempHDBSCANfile.txt"
tempParamFile    = "TempHDBSCANparameters.txt"


def _counts_from_positive_labels(labels: np.ndarray) -> np.ndarray:
    #"""
    #Equivalent to MATLAB accumarray(clusterLabels(clusterLabels>0), 1)
    #"""
    pos = labels[labels > 0]
    if pos.size == 0:
        return np.array([], dtype=int)
    m = int(pos.max())
    counts = np.bincount(pos, minlength=m + 1)  # index 0 unused
    return counts[1:]  # drop 0


# ================================
# 2) LOAD DATA
# ================================
print("Loading data...")
#raw_mat = loadmat(rawDataFile)
#pred_mat = loadmat(predictionFile)

gt=np.loadtxt(rawDataFile)
pred_mat = loadmat(predictionFile)
predictions = pred_mat['predictions']
pred_n = np.round(predictions[0][0])



# KMeans over coordinates
kmeans = KMeans(n_clusters=int(pred_n), n_init="auto", random_state=0)
kmeans.fit(gt[:, :3])
C = kmeans.cluster_centers_


# Compute distances to centroids, assign closest, mark outliers by threshold
D = cdist(gt[:, :3], C, metric="euclidean")
minDist = D.min(axis=1)
idx = D.argmin(axis=1)


clusterLabels = idx.astype(int) + 1  # make them 1..K like MATLAB
clusterLabels[minDist > threshold] = -1


counts = _counts_from_positive_labels(clusterLabels)
label = clusterLabels.copy()
print(f"Preliminary clustering complete. Found {len(counts)} clusters.")

# ================================
# 4) DETERMINE HDBSCAN PARAMETERS
# ================================
if counts.size == 0:
    # no clusters survived the threshold; nothing to refine
    print("No valid clusters from K-Means. Exiting early.")
    hdbscanCluster_CNN: List[Cluster] = []
else:
    m = int(np.min(counts))            # min_cluster_size
    n = int(np.round(0.1 * m))         # min_samples
    print(f"HDBSCAN parameters: min_cluster_size = {m}, min_samples = {n}")

# ================================
# 5) HDBSCAN 
# ================================
import sys
import hdbscan
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Save the first 3 columns of gt to a text file with space delimiter
np.savetxt(tempDataFile, gt[:, :3], delimiter=' ', fmt='%.6f')

# Write m and n to a parameter file
with open(tempParamFile, 'w') as f:
    f.write(f"{m}\n")
    f.write(f"{n}")

    
X=np.genfromtxt(tempDataFile)
Y=np.genfromtxt(tempParamFile)
MinClusterSize=int(Y[0])
MinSamples=int(Y[1])

clusterer = hdbscan.HDBSCAN(min_cluster_size=MinClusterSize,min_samples=MinSamples,
                            cluster_selection_method='eom',approx_min_span_tree=False,core_dist_n_jobs=1)
clusterer.fit(X)
tree=clusterer.condensed_tree_.plot(select_clusters=True,selection_palette=sns.color_palette())
np.savetxt('Labels.txt',clusterer.labels_, fmt='%d',comments='')
np.savetxt('Persistence.txt',clusterer.cluster_persistence_,comments='')
np.savetxt('Probabilities.txt',clusterer.probabilities_,comments='')  


# ================================
# 6) PROCESS HDBSCAN RESULTS (native)
# ================================

# Load data from text files
hdbscan_labels = np.loadtxt('Labels.txt')
hdbscan_persistence = np.loadtxt('Persistence.txt')
hdbscan_probabilities = np.loadtxt('Probabilities.txt')
PositionDataset = np.loadtxt(tempDataFile)
SiZePositionDatasetCol=PositionDataset.shape[1] 

import numpy as np

NoiseLabel = -1
UniqueLabels = np.unique(hdbscan_labels)

hdbscanCluster = []

# Determine if noise label is present
has_noise = NoiseLabel in UniqueLabels

# Adjust loop range depending on presence of noise
loop_range = range(len(UniqueLabels) - 1) if has_noise else range(len(UniqueLabels))

for i in loop_range:
    label = i
    # Find indices where hdbscan_labels == i
    row_indices = np.where(hdbscan_labels == label)[0]

    cluster_info = {
        'labels': label,
        'probabilities': hdbscan_probabilities[row_indices],
        'persistence': hdbscan_persistence[i],
        'atomPositions': PositionDataset[row_indices, :SiZePositionDatasetCol],
        'clustersize': len(row_indices)
    }

    hdbscanCluster.append(cluster_info)

### Step 6 Postprocessing <a class="anchor" id="step6"></a>
The following python script demonstrates how to compute number density, cluster radius, and volume fraction using the clusters structure returned by processHDBSCANResults.

In [ ]:
import numpy as np

# ================================
# Post-Processing of Cluster Properties
# ================================

# INPUTS:
# hdbscanCluster - list of dictionaries from previous step
# subvolume_size - volume of the analyzed region (in nm^3)
# PositionDataset - all points in the analyzed region (Nx3 matrix)

# ---- Parameters ----
Q = 0.36 # Detection efficiency
rho = 84.3 # Atomic density of alpha-Fe (atoms/nm^3)
CrPercent = 0.09 # Cr percentage in the raw 3D data (e.g., 9%)
subvolume_size=40^3 # subvolume size in nm^3

# ---- Number Density (clusters per m^3) ----
num_clusters = len(hdbscanCluster)
number_density = num_clusters / subvolume_size *1e27 # clusters per m^3

# ---- Cluster Radius for Each Cluster (nm) ----
cluster_radii = np.zeros(num_clusters)
for i, cluster in enumerate(hdbscanCluster):
 num_atoms = cluster['clustersize'] # Number of atoms in the cluster
 cluster_radii[i] = ((3 * num_atoms) / (4 * np.pi * rho * Q)) ** (1/3)

# ---- Volume Fraction of Clusters ----
total_cluster_atoms = sum(cluster['clustersize'] for cluster in hdbscanCluster)
total_atoms = PositionDataset.shape[0]
volume_fraction = total_cluster_atoms / (total_atoms / CrPercent)

# ---- Display Results ----
print(f"Number Density: {number_density:.4e} clusters/m^3")
print(f"Average Cluster Radius: {np.mean(cluster_radii):.3f} nm")
print(f"Volume Fraction: {volume_fraction:.4f}")

A simple python code to visualize the clustersusing different colors.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Example: replace this with your actual data
# data = np.loadtxt('your_data.txt')  # or however you load your nx4 matrix
# Assuming data is an (n x 4) numpy array
# Columns: x, y, z, cluster_label
# Example dummy data:
data = np.loadtxt('example_plot.txt')

def plot_clusters(data):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    x, y, z = data[:, 0], data[:, 1], data[:, 2]
    labels = data[:, 3].astype(int)

    for cluster_id in np.unique(labels):
        mask = labels == cluster_id
        ax.scatter(x[mask], y[mask], z[mask], label=f'Cluster {cluster_id}', s=20)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    # ax.legend()
    plt.title('3D Cluster Visualization')
    plt.show()

# Call the function with your data
plot_clusters(data)